## CSCE 676 :: Data Mining and Analysis :: Texas A&M University :: Fall 2019


# Homework 4

- **100 points [5% of your final grade]**
- **Due December 1 by 11:59pm**

*Submission instructions:* You should post your notebook to ecampus (look for the homework 4 assignment there). Please name your submission **your-uin_hw4.ipynb**, so for example, my submission would be something like **555001234_hw4.ipynb**. Your notebook should be fully executed when you submit ... so run all the cells for us so we can see the output, then submit that. 

## (40 points) Part 1: Bloom Filters

For this first part, you are provided a csv file containig around 1 million of user-movie-rating interactions (already sorted by timestamp). You should implement the Counting Bloom Filter algorithm, which stores counts rather then binary indocators in the hash table so that the algorithm can handle deletions.

In your implementation, use the movieId as the key k for the hash function, and the hash function as h_i(k) = ((k % m) + 2^i) % m, where h_i(k) is the i-th hash function, i starts from 0 to r-1, r is the number of hash functions, m is the size of the hash table. Implement your Counting Bloom Filter as a class with r and m as two variables that will be set values when the class is instantiated.

# https://github.com/nitsnwits/simple-bloom-filter/blob/master/src/countingBloomFilter.py 

In [131]:
import numpy as np
import pandas as pd 

a = pd.read_csv('MovieLens_ratings.csv')

a.shape

# size of the hash table
m = 10000

# number of the hash function 
r =5 
 
# Class
class Counting_Bloom_Filter():    
    def __init__(self, m, r):
        self.m = m
        self.r = r

    def empty_hash_table(self):
        return [0 for n in range(self.m)]
         
    def add_to_bloom_filter(self, hash_table, k, r):
        for i in range(r): 
            
            index = (    ( k % self.m )   + 2**i    )   %   self.m
           
            hash_table[index] += 1 
    
        return hash_table 
    
    def check_existence(self, hash_table, k, r):  
 
        for i in range(r):             
            index = (    ( k % self.m )   + 2**i    )   %   self.m
            if(hash_table[index] == 0):
                return False
        
        return True

    def delete_movie_if_exist(self, hash_table, k, r):  

        if(self.check_existence(hash_table, k, r)): 
        
            for i in range(r):             
                index = (    ( k % self.m )   + 2**i    )   %   self.m
                hash_table[index] -= 1
 
        return hash_table



Now, we provide the class to read data from MovieLens_rating.csv. Run the cell below before you continue the homework, which will instanciate a variable dataStream, and you will use it to read data in the following questions. 

In [44]:
import pandas as pd

class DataStream():
    def __init__(self):
        self._df = pd.read_csv('MovieLens_ratings.csv')
        self._pointer = -1
        
    def read_one(self):
        self._pointer += 1
        if self._pointer < len(self._df):
            return {'userId': self._df.at[self._pointer, 'userId'], 
                    'movieId': self._df.at[self._pointer, 'movieId']}
        else:
            return None
    
    def reset(self):
        self._pointer = -1

dataStream = DataStream()

256

After run the above cell, now you need to call dataStream.read_one() for 300,000 times to read in 300,000 records, and maintain one Counting Bloom Filter. Here, set the number of hash functions as 10, and the hash table size as 500.

**!!NOTE!! You should have a loop for 300,000 times, and in each iteration, call dataStream.read_one() first, then update your filter based on the current data sample.**

In [138]:
# Creat Object
Bloom = Counting_Bloom_Filter(500,10)
# print("Bloom.m = " + str(Bloom.m))
# print("Bloom.r = " + str(Bloom.r))

# Creat empty hash table    
h_table = Bloom.empty_hash_table()

all_movie = []

for i in range(300000): 
 
    # Generate one record 
    one_record = dataStream.read_one()
    
    k = one_record['movieId']

    # Delete if exist 
    if(Bloom.check_existence(h_table, k, r)):
        Bloom.delete_movie_if_exist(h_table, k, r)
    
    h_table = Bloom.add_to_bloom_filter(h_table, k, Bloom.r) 

h_table

[3193,
 2365,
 2825,
 3341,
 2686,
 2767,
 2911,
 1587,
 3203,
 3615,
 1837,
 2768,
 4194,
 3179,
 2735,
 2911,
 3020,
 3215,
 3844,
 3646,
 1623,
 3849,
 2887,
 3184,
 2967,
 2759,
 2920,
 1754,
 1745,
 3527,
 2058,
 2472,
 4411,
 3780,
 3105,
 3190,
 3804,
 2785,
 2976,
 3648,
 4520,
 3235,
 2125,
 2508,
 5091,
 3279,
 3018,
 4089,
 3320,
 4443,
 2288,
 3880,
 3287,
 3234,
 2654,
 2532,
 3026,
 2373,
 2711,
 3633,
 3549,
 3509,
 2056,
 2624,
 4001,
 2918,
 2929,
 2484,
 2573,
 2320,
 2363,
 2239,
 2279,
 2481,
 3127,
 2505,
 3063,
 2587,
 2223,
 2004,
 2208,
 2658,
 3392,
 3704,
 3161,
 3896,
 1557,
 2352,
 1832,
 2515,
 2535,
 3303,
 4653,
 3115,
 2199,
 1681,
 3906,
 2510,
 2860,
 3292,
 3016,
 3505,
 2852,
 3199,
 2746,
 4172,
 2556,
 2236,
 3225,
 3250,
 2367,
 2887,
 4189,
 3631,
 4088,
 2363,
 2990,
 2939,
 2173,
 3071,
 3839,
 2528,
 4738,
 2370,
 2692,
 3756,
 3375,
 2995,
 3490,
 4826,
 2410,
 2504,
 2945,
 2997,
 2370,
 2859,
 3092,
 3526,
 1815,
 2457,
 4244,
 2192,
 3313,

### Now, read 100,000 records more, and for each of them, check whether the movie has already been seen during the first 300,000 records based on your Counting Bloom Filter. Output the number of records whose movies are seen before, and the number of records whose movies are not seen before. (During this process, do not update the filter).

In [139]:
seen_m = 0 
not_seen_m = 0

for i in range(10000): 
 
    one_record = dataStream.read_one()
    
    k = one_record['movieId']
 
    flag_seen = Bloom.check_existence(h_table, k, r)  
    
    if(flag_seen): 
        seen_m += 1
    else: 
        not_seen_m +=1
    
print(seen_m)
print(not_seen_m)

10000
0


### Then, read 100,000 records more, and for each of them, check if the movie is in the filter or not, if it is, then delete the movie from the filter, otherwise do nothing.

In [140]:
for i in range(10000): 
    
    # Generate one record 
    one_record = dataStream.read_one()
    k = one_record['movieId']
 
    h_table = Bloom.delete_movie_if_exist(h_table, k, r)  

h_table 

[3145,
 2230,
 2708,
 3229,
 2583,
 2636,
 2811,
 1488,
 3061,
 3510,
 1727,
 2654,
 4088,
 3042,
 2581,
 2798,
 2908,
 3079,
 3712,
 3547,
 1492,
 3711,
 2763,
 3065,
 2879,
 2652,
 2817,
 1668,
 1596,
 3330,
 1940,
 2334,
 4298,
 3679,
 3033,
 3034,
 3630,
 2637,
 2857,
 3560,
 4409,
 3152,
 2025,
 2339,
 4967,
 3151,
 2950,
 4006,
 3227,
 4368,
 2198,
 3770,
 3158,
 3129,
 2540,
 2434,
 2925,
 2278,
 2647,
 3537,
 3452,
 3448,
 1977,
 2549,
 3937,
 2882,
 2835,
 2421,
 2466,
 2253,
 2296,
 2148,
 2171,
 2397,
 3004,
 2409,
 2990,
 2521,
 2136,
 1913,
 2106,
 2527,
 3223,
 3611,
 3049,
 3799,
 1429,
 2264,
 1678,
 2380,
 2396,
 3180,
 4536,
 2993,
 2055,
 1556,
 3727,
 2319,
 2729,
 3189,
 2907,
 3370,
 2732,
 3097,
 2661,
 4027,
 2453,
 2130,
 3136,
 3107,
 2251,
 2770,
 4017,
 3520,
 3974,
 2246,
 2857,
 2853,
 2013,
 2963,
 3771,
 2424,
 4678,
 2321,
 2581,
 3646,
 3256,
 2908,
 3404,
 4741,
 2315,
 2403,
 2852,
 2906,
 2327,
 2812,
 3023,
 3460,
 1762,
 2395,
 4159,
 2119,
 3235,

### At last, read 100,000 records more, and for each of them, check whether the movie is in the filter or not. Output the number of records whose movies are seen before, and the number of records whose movies are not seen before. (During this process, do not update the filter).

In [141]:
seen_m = 0 
not_seen_m = 0

for i in range(10000): 
 
    one_record = dataStream.read_one()
    
    k = one_record['movieId']
 
    flag_seen = Bloom.check_existence(h_table, k, r)  
    
    if(flag_seen): 
        seen_m += 1
    else: 
        not_seen_m +=1
    
print(seen_m)
print(not_seen_m)

10000
0


### Before we go to the next part, let's reset the dataStream and repeat the above process to explore the impact of r and m. 
### 1. You need to call dataStream.reset() to reset the reader, then call dataStream.read_one() for 300,000 times and build your filter. 
### 2. Then, call dataStream.read_one() for another 300,000 times, and for each of the data sample, check whether the movie is in the first 300,000 records or not based on your filter, and report the False Positive Rate. 
### 3. Try different r and m, **plot figures (one or two)** to show how false positive rate changes with different r and m, and **report the best pair of parameters** you think and **discuss why**.

In [126]:
dataStream.reset()

In [124]:
Bloom = Counting_Bloom_Filter(50000,10)

print("Bloom.r = " + str(Bloom.r))
print("Bloom.m = " + str(Bloom.m))

h_table_new = Bloom.empty_hash_table()
 
for i in range(300000): 
 
    one_record = dataStream.read_one()
    
    k = one_record['movieId']
  
    h_table_new = Bloom.add_to_bloom_filter(h_table, k, Bloom.r) 

h_table_new

Bloom.r = 5
Bloom.m = 500


[2788,
 8644,
 9786,
 9924,
 7087,
 11013,
 6769,
 8061,
 8997,
 9497,
 8621,
 6491,
 7797,
 9694,
 8790,
 7779,
 6726,
 10350,
 9735,
 9363,
 7953,
 9876,
 10043,
 9271,
 6804,
 10013,
 9231,
 9033,
 9974,
 14385,
 9015,
 9210,
 8854,
 14011,
 8419,
 12332,
 13800,
 11408,
 10946,
 5460,
 12656,
 9173,
 9130,
 13876,
 9720,
 9411,
 5648,
 8744,
 9503,
 9056,
 8143,
 9375,
 10186,
 9005,
 9563,
 9354,
 7478,
 6849,
 6855,
 5685,
 7629,
 5302,
 6030,
 6154,
 6778,
 5077,
 11151,
 4176,
 8432,
 4341,
 6713,
 7259,
 6233,
 6697,
 9141,
 7530,
 4829,
 5889,
 6597,
 6606,
 7396,
 9140,
 10839,
 6485,
 7298,
 8003,
 8065,
 6784,
 10230,
 11104,
 9602,
 8440,
 8711,
 8485,
 10717,
 9807,
 13517,
 12146,
 11841,
 8379,
 7658,
 10794,
 9100,
 9699,
 6459,
 10994,
 8386,
 6395,
 7715,
 12599,
 7891,
 11350,
 15764,
 10397,
 10942,
 8239,
 10762,
 5736,
 11797,
 8034,
 5780,
 7350,
 4530,
 4070,
 8262,
 5828,
 10581,
 7753,
 5378,
 8886,
 7325,
 5786,
 5608,
 7252,
 5237,
 4211,
 6340,
 5696,
 45

In [130]:
# check false-positive
original_data = pd.read_csv('MovieLens_ratings.csv')

data_1 = original_data.loc[0:29999,'movieId']
data_test = []
data_2 = original_data.loc[300000:599999,'movieId']

print(data_1.shape)
print(data_2.shape)

exist_count = 0

for i in data_2: 
    if i in data_1:
        exist_count += 1

print(exist_count)
 
##########################################################
 
m_exist = 0

for i in range(300000): 
 
    one_record = dataStream.read_one()
    
    k = one_record['movieId']
  
    check_flag = Bloom.check_existence(h_table, k, Bloom.r) 

    if(check_flag): 
        m_exist += 1
m_exist
 

(30000,)
(300000,)
300000


TypeError: 'NoneType' object is not subscriptable

(300001,)
(300001,)


300001

In [ ]:
# plot figure(s)


`Put your discussion here.`
### Discussion --> Here is the insight. We have 1 million data - movieId. First, we use 300,000 movieId to build the Counting Bloom Filter, and then test with another 300,000. 

### The problem in the question is: In the CVS data, the later 300K are all exist in the former 300K data. No matter how I tune the parameter r or m, the filter already include every movieId - which we have no way to test the false positive! Because the false positive are always 0



## (30 points) Part 2: Determine approximately the top-5 movies

In this part, you need to first reset the dataStream, then read the whole file by dataStream.read_one() util it returns None. You need to implement the reservoir sampling algorithm to determine approximately the top-5 most frequent movies. 

First you need to implement a function add_reservoir(reservoir, movieId, max_size) that adds a movie to the reservoir, maintaining its size. If the reservoir is already of size max_size, a random movie is selected and evicted before adding the new movie. It is important to evict an old movie before adding the new item.


dataStream.reset()

In [4]:
def add_to_reservoir(reservoir, movieId, max_reservoir_size, i):
    # YOUR CODE HERE

    K = max_reservoir_size
    if( len(reservoir) < max_reservoir_size  ): 
        reservoir.append(movieId)
    else: 
        s = int(random.random() * i)
        if s < K:
                reservoir[ s ] = movieId
            

In [50]:
dataStream.reset()
if dataStream == None: 
    print("Fale")

Iterate through the file using the reservoir sampling method with maximum size of 1,000. 

In [6]:
import random 

dataStream.reset()

reservoir_size = 1000
reservoir = []

i = 0

dataSample = dataStream.read_one()

while not dataSample == None:
    i += 1
    dataSample = dataStream.read_one()
    movieId = dataSample['movieId']
    add_to_reservoir(reservoir, movieId, reservoir_size,i)
    
    if i % 100000 == 0: 
        print(i)
     
print("Number of records seen    : %d" % i)
print("Number of records sampled : %d" % len(reservoir) )

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000


TypeError: 'NoneType' object is not subscriptable

Write code to list the top-5 most frequent movies found by looking at frequencies in the reservoir. If you see a movie C times in the reservoir, you can estimate the query appears C x N / reservoir_size times in the actual sample. **You need to join the resulting top-5 movie ids with the file movies.dat to show the top-5 movies by their names.**

In [126]:
from collections import Counter
def counter(arr):
    return Counter(arr).most_common(5) # 返回出现频率最高的两个数

top_5 = counter(reservoir)

top_5_cleaned = []

for i in range(5): 
     top_5_cleaned.append(top_5[i][0])
        
top_5_cleaned       



[1136, 2762, 1210, 1721, 185]

In [129]:
import pandas as pd    

with open('movies.dat','r',encoding="utf8", errors='ignore') as f:
    next(f) # skip first row
    df = pd.DataFrame(l.rstrip().split() for l in f)

df.shape

df_x = df.iloc[:,0]

df_x.to_csv(r'pandas.txt', header=None, index=None, sep=' ', mode='a')

a = []
b = [] 

# clearn the variable "str"
# del str

for i in range(len(df_x)): 
    str_r = df_x[i]
    strlist = str_r.split('::') # 用逗号分割str字符串，并保存到列表
    a.append(strlist[0])
    b.append(strlist[1])

'''
print(a[3000])
print(b[3000])

print(str(123))
'''

for i in top_5_cleaned: 
    print(i)
    index = a.index(str(i))
    print(b[index])

1136
Monty
2762
Sixth
1210
Star
1721
Titanic
185
Net,


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  # This is added back by InteractiveShellApp.init_path()


Use dataStream._df to get the DataFrame variable of the whole MovieLens_ratings.csv, and compute the top-5 most frequent movies by the dataframe. Compare with the approximating results by reservoir sampling algorithm, **discuss your observations**.

In [159]:
df_all_movies = dataStream._df.iloc[:,1]

print(df_all_movies.shape)

from collections import Counter

def counter(arr):
    return Counter(arr).most_common(5) # 返回出现频率最高的5个数

top_5 = counter(df_all_movies)

for i in range(5): 
     print(top_5[i][0])
         

(1000209,)
2858
260
1196
1210
480


`Put your discussion here.` We can see the most frequent movieId never changed. However, from the reservior, there is one movie tagged.  

At last, repeat the process above and try different reservoir_size, compare the correpsonding results with ground truth, **report the best parameter you think** and **discuss the reason**.

In [5]:
import random 

for i in range(10):  
    
    dataStream.reset()

    reservoir_size = 1000
    reservoir = []

    i = 0

    dataSample = dataStream.read_one()

    while not dataSample == None:
        i += 1
        dataSample = dataStream.read_one()
        movieId = dataSample['movieId']
        add_to_reservoir(reservoir, movieId, reservoir_size,i)

        if i % 100000 == 0: 
            # print(i)
            None

        if i == 1000000: 
            break 

    print("Number of records seen    : %d" % i)
    print("Number of records sampled : %d" % len(reservoir) )


    from collections import Counter
    def counter(arr):
        return Counter(arr).most_common(5) # 返回出现频率最高的两个数

    top_5 = counter(reservoir)

    top_5_cleaned = []

    for i in range(5): 
         top_5_cleaned.append(top_5[i][0])

    print(top_5_cleaned)




Number of records seen    : 1000000
Number of records sampled : 1000
[593, 2571, 318, 608, 1136]
Number of records seen    : 1000000
Number of records sampled : 1000
[1210, 608, 1198, 3175, 110]
Number of records seen    : 1000000
Number of records sampled : 1000
[858, 1196, 1183, 1198, 589]
Number of records seen    : 1000000
Number of records sampled : 1000
[2858, 593, 2324, 110, 1704]
Number of records seen    : 1000000
Number of records sampled : 1000
[223, 1198, 527, 110, 919]
Number of records seen    : 1000000
Number of records sampled : 1000
[2716, 2571, 2858, 1371, 1580]
Number of records seen    : 1000000
Number of records sampled : 1000
[1219, 527, 2028, 3471, 2528]
Number of records seen    : 1000000
Number of records sampled : 1000
[589, 1580, 1196, 1198, 3160]
Number of records seen    : 1000000
Number of records sampled : 1000
[318, 2791, 480, 1265, 1206]
Number of records seen    : 1000000
Number of records sampled : 1000
[1387, 919, 1356, 1197, 3578]


In [23]:
import random 

dataStream.reset()

reservoir_size = 80000
reservoir = []

i = 0

dataSample = dataStream.read_one()

while not dataSample == None:
    i += 1
    dataSample = dataStream.read_one()
    movieId = dataSample['movieId']
    add_to_reservoir(reservoir, movieId, reservoir_size,i)

    if i % 100000 == 0: 
        # print(i)
        None

    if i == 1000000: 
        break 

print("Number of records seen    : %d" % i)
print("Number of records sampled : %d" % len(reservoir) )


from collections import Counter
def counter(arr):
    return Counter(arr).most_common(10) # 返回出现频率最高的两个数

top_5 = counter(reservoir)

top_5_cleaned = []

for i in range(10): 
     top_5_cleaned.append(top_5[i][0])

        
        
print("reservoir_size = " + str(reservoir_size))
print(top_5_cleaned)




Number of records seen    : 1000000
Number of records sampled : 80000
reservoir_size = 80000
[2858, 1198, 1210, 1196, 260, 110, 2762, 1197, 608, 2571]


# 2858 260 1196 1210 480

`Put your discussion here.`
size = 500
Top 10 movie

*2858, 2571, 527, *480, *260, *1210, *1196, 589, 2997, 1259
2858 260 1196 1210 480

size = 500
Top 10 movie
590, 2797, 2948, 11, 780, 2514, 3261, 3893, *1196, 1029
2858 260 1196 1210 480

size = 2000
Top 10 movie
[111, 2791, 593, 541, 924, 2640, *2858, 1183, 110, 1617]
2858 260 1196 1210 480

reservoir_size = 20000
*2858, 1097, *260, 1198, 2571, *1196, 110, *1210, 2028, 858]
2858 260 1196 1210 480

reservoir_size = 50000
*260, *2858, *1196, 593, 2571, *480, 110, 1580, 608, *1210
2858 260 1196 1210 480

reservoir_size = 100000
[*2858, *260, *1210, *1196, *480, 2571, 1580, 593, 1270, 2028]
2858 260 1196 1210 480
 
`As we can see here, when the reservior size is 100,000, we can have the best prediction. However, even we only have 20,000, we probably also can find all top_5 movies in top 10`

## (30 points) Part 3: Determine approximately the number of users.

In this part, you need to estimate the number of distinct users without creating a hash table with users, but instead, you should use the Flajolet-Martin probabilistic counting method.

First you need to reset dataStream.

In [24]:
dataStream.reset()

Use this function to count trailing zeroes in the binary representation of a number.

In [26]:
def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

Use the function below to generate a random hash function. Note this generates a function, so you can do hash_function = random_hash_function() and then call hash_function(x) to compute the hash value of x.

In [27]:
def random_hash_function():
    salt = random.random()
    return lambda string: hash(string + str(salt))

In [32]:
print(count_trailing_zeroes(6))

h_function = random_hash_function()

print(h_function("555"))

1
1889979419633121563


Now implement your Flajolet-Martin probabilistic counting algorithm and perform 10 passes over the file, create a new hash function and use it to hash userids. Keep the maximum number of trailing zeroes seen in the hash value of a userid.

Estimate on pass 1: 16384
Estimate on pass 2: 16384
Estimate on pass 3: 16384
Estimate on pass 4: 16384
Estimate on pass 5: 16384
Estimate on pass 6: 16384
Estimate on pass 7: 16384
Estimate on pass 8: 16384
Estimate on pass 9: 16384
Estimate on pass 10: 16384
* Average of estimates: 16384.0000
* Median  of estimates: 16384.0000


In [ ]:

dataStream.reset()

df_all_user = dataStream._df.iloc[:,0]

print(df_all_user.shape)

unique_list = [] 

for x in df_all_user: 
    if x not in unique_list: 
        unique_list.append(x) 
       


print(len(unique_list))


 

In [ ]:
estimate(df_all_user,3)

In [75]:
import os
import random
import mmh3
import re
import numpy as np


def median(lst):
    even = (0 if len(lst) % 2 else 1) + 1
    half = (len(lst) - 1) / 2
    half = int(half)
    return sum(sorted(lst)[half:half + even]) / float(even)


class HashFunction:
    def __init__(self, seed):
        self.func = lambda x: (mmh3.hash(x, seed=seed) % 2 ** 20) + 1
        self.max_tail_count = 0


class HashGroup:
    def __init__(self, num_hashes_in_group=10):
        self.hash_functions = []
        for i in range(num_hashes_in_group):
            self.hash_functions.append(HashFunction(random.randint(1, 10000)))


class FlajoletMartinCounter:
    def __init__(self, num_hash_groups=10, num_hashes_in_group=10):
        self.hash_groups = []
        for i in range(num_hash_groups):
            self.hash_groups.append(HashGroup(num_hashes_in_group))

    def tail_length(self, num):
        """Counts the number of trailing 0 bits in num."""
        if num == 0:
            return 32  # Assumes 32 bit integer inputs!
        p = 0
        while (num >> p) & 1 == 0:
            p += 1
        return p

    def process(self, element):
        for g in self.hash_groups:
            for f in g.hash_functions:
                h = f.func(element)
                tail_count = self.tail_length(h)
                if tail_count > f.max_tail_count:
                    f.max_tail_count = tail_count

    def give_estimate(self):
        estimate_pr_group = []
        for g in self.hash_groups:
            estimate_in_group = []
            for f in g.hash_functions:
                estimate_in_group.append(2 ** f.max_tail_count)
            estimate_pr_group.append(np.mean(estimate_pr_group))
        return 1.0 * np.median(estimate_in_group)

import numpy as np

number_of_passes = 10

estimates = []
 
for i in range(number_of_passes):
    
   # using Python set to estimate the number of distinct words in a Shakespeare text
    Quotes = set()

    # Using FM to estimate the number of distinct words in a Shakespeare text
    FM = FlajoletMartinCounter(num_hash_groups=10, num_hashes_in_group=10)

    for line in df_all_user[0:1000000]:
                FM.process(str(line))
                Quotes.add(line)

    print("Estimate on pass = " + str(FM.give_estimate()) )            


    estimates.append(FM.give_estimate())
         
    
print("* Average of estimates: %.4f" % np.mean(estimates))
print("* Median  of estimates: %.4f" % np.median(estimates))

Estimate on pass = 6144.0
Estimate on pass = 6144.0
Estimate on pass = 8192.0
Estimate on pass = 8192.0
Estimate on pass = 6144.0
Estimate on pass = 8192.0
Estimate on pass = 8192.0
Estimate on pass = 4096.0
Estimate on pass = 4096.0
Estimate on pass = 8192.0
* Average of estimates: 6758.4000
* Median  of estimates: 7168.0000
